In [1]:
import os;
import json;
import builtins
import duckdb
import pandas as pd;
import matplotlib.pyplot as plt


def get_dataset_from_testcase(run):
    return run[0:-2]

def get_dataset_size(dataset):
    if dataset == "osm" or dataset == "books":
        return 800_000_000
    else:
        return 200_000_000

dir = '/home/chesetti/Repos/KVector_Merge/sponge/join_all'
runs = []
for test_case in os.listdir(dir):
    if test_case == 'build':
        continue
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results', 'run')):
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results', 'run', run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        run['dataset_size'] = get_dataset_size(run['dataset'])
        runs.append(run)
df = pd.json_normalize(runs)
print(df.columns)

FileNotFoundError: [Errno 2] No such file or directory: '/home/chesetti/Repos/KVector_Merge/sponge/join_all'

In [ ]:
def get_index_type(index):
    if "btree" in index:
        return "BTREE"
    if "pgm" in index:
        return "PGM"
    return "NA"

def get_index_variant(index):
    if index == "sort_join" or index == "hash_join":
        return "NA"
    pos = index.find("_")
    return index[pos+1:-1]

df["threads"] = df["spec.num_threads"]
df["epsilon"] = df["spec.index.epsilon"]
df["duration_sec"] = df["result.duration_ns"] / (1000000000)
df["ratio"] = df["spec.common_key"]
df["thput"] = (df["result.num_output_keys"] / df["ratio"]) / (df["result.duration_ns"] / (1000000000))
df["algo"] = df["spec.algo_name"]
df["join_algo"] = df["spec.algo"]
df["index_type"] = df["spec.algo_name"].map(lambda x: get_index_type(x))
df["index_variant"] = df["spec.algo_name"].map(lambda x: get_index_variant(x))
df["inner_bytes_fetched"] = df["result.inner_total_bytes_fetched"]
display(df["algo"].unique())
display(df["dataset"].unique())
display(df["index_type"].unique())


array(['lsj_pgm4096', 'lsj_sampledflatpgm4096', 'hash_join',
       'lsj_sampledflatpgm1024', 'inlj_flatpgm256', 'lsj_btree4096',
       'sort_join', 'inlj_sampledflatpgm4096', 'inlj_btree4096',
       'lsj_flatpgm4096', 'lsj_btree1024', 'inlj_pgm256', 'lsj_btree256',
       'inlj_pgm4096', 'inlj_sampledflatpgm1024', 'lsj_pgm256',
       'inlj_sampledflatpgm256', 'inlj_btree1024', 'lsj_flatpgm1024',
       'lsj_pgm1024', 'inlj_flatpgm4096', 'lsj_flatpgm256',
       'inlj_flatpgm1024', 'inlj_pgm1024', 'inlj_btree256',
       'lsj_sampledflatpgm256'], dtype=object)

array(['wiki', 'uniform_dense', 'normal', 'uniform_sparse', 'lognormal',
       'fb'], dtype=object)

array(['PGM', 'NA', 'BTREE'], dtype=object)

In [ ]:
# SingleThread, HJ vs SJ vs INLJ(BTree256) vs INLJ(PGM256) vs LS(BTree256) vs LS(PGM256)
def plot_dataset_join_duration(dataset):
    rows = duckdb.sql(
        "SELECT ratio, threads, algo, MEDIAN(duration_sec) as d, MEDIAN(thput) as t FROM df " 
        "WHERE" 
        "   (algo='hash_join' OR algo='sort_join' OR algo='inlj_btree256' OR algo='inlj_pgm256' OR "
        "   algo='lsj_btree256' OR algo='lsj_flatpgm256')" 
        "   AND threads=1"
        f"   AND dataset='{dataset}'"
        "GROUP BY dataset, ratio, threads, algo"
    ).df()
    return(rows.pivot(index='ratio', values='d', columns=['algo']))

real_datasets = ['fb', 'wiki', 'osm', 'books']
synth_datasets = ['uniform_dense', 'uniform_sparse', 'normal', 'lognormal']

for dataset in real_datasets:
    display(dataset)
    display(plot_dataset_join_duration(dataset))

for dataset in synth_datasets:
    display(dataset)
    display(plot_dataset_join_duration(dataset))

'fb'

algo,hash_join,inlj_btree256,inlj_pgm256,lsj_btree256,lsj_flatpgm256,sort_join
ratio,,,,,,
1,79.785572,53.723555,33.464064,52.649181,26.987002,25.162671
10,53.982077,5.690409,4.085079,5.636519,3.016305,7.079598
100,44.572595,1.196704,0.943092,1.200733,0.740194,5.148352
1000,20.136223,0.225369,0.225207,0.227195,0.218850,4.931778


'wiki'

algo,hash_join,inlj_btree256,inlj_pgm256,lsj_btree256,lsj_flatpgm256,sort_join
ratio,,,,,,
1,32.578296,23.270138,14.453964,22.860397,12.128019,11.324560
10,24.205699,4.822314,3.162925,4.746160,2.529570,4.194891
100,20.306184,0.869956,0.698924,0.867694,0.503725,2.513819
1000,8.994534,0.208226,0.180925,0.208636,0.173009,2.319318


'osm'

...

algo,hash_join,inlj_btree256,inlj_pgm256,lsj_btree256,lsj_flatpgm256,sort_join
ratio,,,,,,
1,79.824059,52.790627,31.543096,51.743549,26.768670,24.984327
10,53.865636,5.705050,3.956811,5.632568,3.010531,7.082598
100,44.547378,1.230551,0.937140,1.231746,0.756002,5.154212
1000,22.689613,0.236518,0.231072,0.237742,0.226380,4.946246


'normal'

algo,hash_join,inlj_btree256,inlj_pgm256,lsj_btree256,lsj_flatpgm256,sort_join
ratio,,,,,,
1,80.190714,53.444584,31.871422,52.369877,26.820202,25.023829
10,53.703619,5.877991,4.001172,5.775892,3.066949,7.252469
100,44.970908,1.265768,0.947298,1.259856,0.766837,5.296070
1000,22.897787,0.240616,0.234836,0.240155,0.230603,5.102566


'lognormal'

algo,hash_join,inlj_btree256,inlj_pgm256,lsj_btree256,lsj_flatpgm256,sort_join
ratio,,,,,,
1,78.720935,52.932648,33.314757,51.819811,25.668690,24.880721
10,53.873789,5.701944,4.224656,5.654821,2.961539,7.065209
100,44.420552,1.213984,1.024411,1.214580,0.755119,5.132633
1000,19.661215,0.231155,0.237930,0.230680,0.228448,4.928607


In [ ]:
# Effect of Epsilon on INLJ (BTree and PGM)
def plot_epsilon_inlj(dataset):
    rows = duckdb.sql(
        "SELECT ratio, algo, join_algo, epsilon, MEDIAN(duration_sec) as d, MEDIAN(thput) as t FROM df " 
        "WHERE" 
        "   join_algo = 'inlj'"
        "   AND threads=1"
        f"   AND dataset='{dataset}'"
        "GROUP BY dataset, ratio, algo, join_algo, epsilon"
    ).df()
    return rows.pivot(index='ratio', values='d', columns=['algo', 'epsilon'])

for dataset in real_datasets:
    display(dataset)
    display(plot_epsilon_inlj(dataset))

for dataset in synth_datasets:
    display(dataset)
    display(plot_epsilon_inlj(dataset))

'fb'

algo,inlj_btree1024,inlj_flatpgm4096,inlj_btree4096,inlj_sampledflatpgm4096,inlj_sampledflatpgm1024,inlj_btree256,inlj_pgm256,inlj_pgm1024,inlj_flatpgm256,inlj_pgm4096,inlj_sampledflatpgm256,inlj_flatpgm1024
epsilon,1024.0,4096.0,4096.0,4096.0,1024.0,256.0,256.0,1024.0,256.0,4096.0,256.0,1024.0
ratio,,,,,,,,,,,,
1,54.588636,33.575393,52.968209,34.713387,35.019604,53.723555,33.464064,33.376552,30.652944,36.123307,38.252170,32.459946
10,5.497925,3.931653,5.307913,4.218466,4.537026,5.690409,4.085079,3.915549,3.843559,4.183998,5.138802,3.819236
100,0.909161,0.995085,0.809161,1.086608,1.176709,1.196704,0.943092,0.970635,0.921725,1.022775,1.276552,0.949179
1000,0.286764,0.384095,0.307143,0.401380,0.300688,0.225369,0.225207,0.277599,0.223829,0.384734,0.252409,0.274128


'wiki'

algo,inlj_sampledflatpgm4096,inlj_pgm4096,inlj_btree4096,inlj_flatpgm256,inlj_pgm256,inlj_pgm1024,inlj_flatpgm4096,inlj_sampledflatpgm1024,inlj_sampledflatpgm256,inlj_btree256,inlj_btree1024,inlj_flatpgm1024
epsilon,4096.0,4096.0,4096.0,256.0,256.0,1024.0,4096.0,1024.0,256.0,256.0,1024.0,1024.0
ratio,,,,,,,,,,,,
1,15.455615,15.827330,21.523201,13.696504,14.453964,14.999373,14.929495,15.174188,15.390420,23.270138,22.577373,14.482200
10,3.422134,3.424825,4.258731,3.041385,3.162925,3.296488,3.234768,3.531694,3.716301,4.822314,4.524638,3.120329
100,0.814383,0.738932,0.615451,0.655918,0.698924,0.713232,0.716621,0.913820,0.978394,0.869956,0.689098,0.706915
1000,0.302543,0.278255,0.181401,0.181253,0.180925,0.224906,0.274055,0.246377,0.216119,0.208226,0.207184,0.216769


'osm'

...

algo,inlj_btree1024,inlj_flatpgm256,inlj_pgm4096,inlj_btree4096,inlj_sampledflatpgm256,inlj_flatpgm1024,inlj_pgm1024,inlj_pgm256,inlj_flatpgm4096,inlj_sampledflatpgm4096,inlj_btree256,inlj_sampledflatpgm1024
epsilon,1024.0,256.0,4096.0,4096.0,256.0,1024.0,1024.0,256.0,4096.0,4096.0,256.0,1024.0
ratio,,,,,,,,,,,,
1,53.698649,29.983271,32.611604,52.300837,33.020995,31.027543,31.160718,31.543096,31.841783,32.114671,52.790627,31.503087
10,5.494754,3.776262,3.856450,5.306251,4.967536,3.747043,3.766312,3.956811,3.804095,3.894787,5.705050,4.152439
100,0.924741,0.917394,0.992095,0.826849,1.310415,0.980722,0.984568,0.937140,0.990347,1.081915,1.230551,1.189227
1000,0.301985,0.228666,0.397535,0.323396,0.256023,0.290428,0.289575,0.231072,0.400153,0.421276,0.236518,0.312569


'normal'

algo,inlj_flatpgm4096,inlj_pgm4096,inlj_sampledflatpgm4096,inlj_sampledflatpgm1024,inlj_pgm1024,inlj_sampledflatpgm256,inlj_flatpgm1024,inlj_btree256,inlj_flatpgm256,inlj_btree4096,inlj_btree1024,inlj_pgm256
epsilon,4096.0,4096.0,4096.0,1024.0,1024.0,256.0,1024.0,256.0,256.0,4096.0,1024.0,256.0
ratio,,,,,,,,,,,,
1,33.292856,35.093939,33.668926,32.114858,33.541749,31.614064,31.539024,53.444584,29.944082,52.923079,54.541270,31.871422
10,4.308542,4.475421,5.003237,4.645596,4.135038,4.970013,3.927536,5.877991,3.806000,5.431510,5.540704,4.001172
100,1.214645,1.222520,1.478316,1.318857,1.045134,1.329774,1.019723,1.265768,0.925729,0.835695,0.937591,0.947298
1000,0.424541,0.431134,0.455504,0.323160,0.299738,0.263512,0.298296,0.240616,0.231377,0.329501,0.304140,0.234836


'lognormal'

algo,inlj_btree1024,inlj_sampledflatpgm256,inlj_pgm1024,inlj_flatpgm256,inlj_btree256,inlj_pgm4096,inlj_pgm256,inlj_sampledflatpgm4096,inlj_flatpgm4096,inlj_btree4096,inlj_sampledflatpgm1024,inlj_flatpgm1024
epsilon,1024.0,256.0,1024.0,256.0,256.0,4096.0,256.0,4096.0,4096.0,4096.0,1024.0,1024.0
ratio,,,,,,,,,,,,
1,53.684522,31.834472,33.354719,30.133240,52.932648,35.825353,33.314757,34.064562,33.705478,52.155415,31.939601,31.595328
10,5.505030,5.153610,4.261129,3.900504,5.701944,4.790636,4.224656,5.560521,4.624012,5.347522,4.967971,4.068290
100,0.911732,1.421147,1.158466,0.979978,1.213984,1.469182,1.024411,1.786009,1.437676,0.812216,1.488593,1.140116
1000,0.303248,0.261101,0.306920,0.231444,0.231155,0.504190,0.237930,0.501250,0.470993,0.314207,0.332593,0.303786


In [ ]:
# Effect of Epsilon on INLJ (BTree and PGM)
def plot_epsilon_lsj(dataset):
    rows = duckdb.sql(
        "SELECT ratio, algo, join_algo, epsilon, MEDIAN(duration_sec) as d, MEDIAN(thput) as t FROM df " 
        "WHERE" 
        "   join_algo = 'lsj'"
        "   AND threads=1"
        f"   AND dataset='{dataset}'"
        "GROUP BY dataset, ratio, algo, join_algo, epsilon"
    ).df()
    return rows.pivot(index='ratio', values='d', columns=['algo', 'epsilon'])

for dataset in real_datasets:
    display(dataset)
    display(plot_epsilon_lsj(dataset))

for dataset in synth_datasets:
    display(dataset)
    display(plot_epsilon_lsj(dataset))

'fb'

algo,lsj_flatpgm4096,lsj_flatpgm1024,lsj_flatpgm256,lsj_btree4096,lsj_sampledflatpgm256,lsj_pgm256,lsj_sampledflatpgm4096,lsj_btree1024,lsj_pgm1024,lsj_pgm4096,lsj_sampledflatpgm1024,lsj_btree256
epsilon,4096.0,1024.0,256.0,4096.0,256.0,256.0,4096.0,1024.0,1024.0,4096.0,1024.0,256.0
ratio,,,,,,,,,,,,
1,29.859236,28.474029,26.987002,51.813857,27.985456,31.933872,30.610882,52.765188,31.524737,35.218450,28.824079,52.649181
10,3.153408,3.010683,3.016305,5.227907,3.105649,3.532868,3.183010,5.427577,3.388625,3.654325,3.054856,5.636519
100,0.610235,0.613775,0.740194,0.811296,0.770686,0.798612,0.574284,0.901017,0.652138,0.626493,0.621495,1.200733
1000,0.275550,0.253435,0.218850,0.315796,0.242841,0.226063,0.273259,0.288444,0.260004,0.277396,0.268029,0.227195


'wiki'

algo,lsj_btree4096,lsj_btree1024,lsj_btree256,lsj_pgm256,lsj_sampledflatpgm1024,lsj_pgm4096,lsj_flatpgm4096,lsj_sampledflatpgm4096,lsj_sampledflatpgm256,lsj_flatpgm256,lsj_flatpgm1024,lsj_pgm1024
epsilon,4096.0,1024.0,256.0,256.0,1024.0,4096.0,4096.0,4096.0,256.0,256.0,1024.0,1024.0
ratio,,,,,,,,,,,,
1,21.121723,22.511216,22.860397,13.852011,12.800034,15.187852,13.654480,13.754466,12.719507,12.128019,12.537260,14.158023
10,4.226802,4.485158,4.746160,2.816370,2.615648,3.071957,2.726041,2.755398,2.579863,2.529570,2.600654,2.864973
100,0.612697,0.676807,0.867694,0.538358,0.460861,0.479688,0.444253,0.446688,0.523436,0.503725,0.451815,0.488916
1000,0.181598,0.206332,0.208636,0.176829,0.178203,0.163821,0.160087,0.168344,0.196053,0.173009,0.173633,0.176845


'osm'

...

algo,lsj_pgm4096,lsj_sampledflatpgm4096,lsj_flatpgm4096,lsj_btree256,lsj_flatpgm256,lsj_sampledflatpgm1024,lsj_btree4096,lsj_btree1024,lsj_flatpgm1024,lsj_pgm1024,lsj_sampledflatpgm256,lsj_pgm256
epsilon,4096.0,4096.0,4096.0,256.0,256.0,1024.0,4096.0,1024.0,1024.0,1024.0,256.0,256.0
ratio,,,,,,,,,,,,
1,31.689784,30.165467,29.883543,51.743549,26.768670,28.602864,51.374448,52.749507,28.348640,30.189751,27.468063,30.333008
10,3.323338,3.234927,3.164637,5.632568,3.010531,3.101642,5.252183,5.417373,3.043242,3.223437,3.103629,3.379430
100,0.611171,0.613100,0.592688,1.231746,0.756002,0.648281,0.816544,0.914489,0.639914,0.651727,0.788451,0.795113
1000,0.290296,0.300379,0.294354,0.237742,0.226380,0.286976,0.322849,0.300796,0.271000,0.270081,0.254252,0.231511


'normal'

algo,lsj_sampledflatpgm1024,lsj_btree4096,lsj_flatpgm4096,lsj_btree256,lsj_flatpgm1024,lsj_pgm1024,lsj_pgm4096,lsj_flatpgm256,lsj_pgm256,lsj_sampledflatpgm4096,lsj_sampledflatpgm256,lsj_btree1024
epsilon,1024.0,4096.0,4096.0,256.0,1024.0,1024.0,4096.0,256.0,256.0,4096.0,256.0,1024.0
ratio,,,,,,,,,,,,
1,28.875777,52.070128,29.770474,52.369877,28.198332,31.409620,33.094355,26.820202,30.336636,29.927734,27.938565,52.972112
10,3.147763,5.288045,3.127781,5.775892,3.026056,3.353272,3.510877,3.066949,3.415590,3.189761,3.168278,5.472135
100,0.643274,0.859088,0.601905,1.259856,0.651120,0.686029,0.662887,0.766837,0.793542,0.609589,0.809134,0.917467
1000,0.290480,0.329414,0.289028,0.240155,0.271792,0.277214,0.296629,0.230603,0.232950,0.293955,0.257686,0.303025


'lognormal'

algo,lsj_pgm4096,lsj_btree256,lsj_flatpgm4096,lsj_sampledflatpgm4096,lsj_btree4096,lsj_sampledflatpgm1024,lsj_sampledflatpgm256,lsj_flatpgm256,lsj_btree1024,lsj_pgm256,lsj_pgm1024,lsj_flatpgm1024
epsilon,4096.0,256.0,4096.0,4096.0,4096.0,1024.0,256.0,256.0,1024.0,256.0,1024.0,1024.0
ratio,,,,,,,,,,,,
1,32.166440,51.819811,28.495432,29.427815,51.340584,28.170980,27.486908,25.668690,52.666640,30.897326,30.556263,27.017295
10,3.398305,5.654821,3.053803,3.124540,5.242098,3.066378,3.102597,2.961539,5.396317,3.471716,3.316870,2.959799
100,0.614113,1.214580,0.579760,0.585797,0.803116,0.638884,0.789848,0.755119,0.905219,0.814019,0.657125,0.623597
1000,0.289814,0.230680,0.284259,0.284786,0.311328,0.287632,0.257202,0.228448,0.295275,0.233523,0.278461,0.274921
